In [1]:
import pandas as pd

In [2]:
df = pd.read_csv("LoanApprovalPrediction.csv")

In [3]:
df['Dependents'].fillna('-',inplace=True)
df['Loan_Amount_Term'].fillna('-',inplace=True)
df['Credit_History'].fillna('-',inplace=True)

C:\Users\Akhilesh\AppData\Local\Temp\ipykernel_34984\2899975127.py:1: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '-' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  df['Dependents'].fillna('-',inplace=True)
C:\Users\Akhilesh\AppData\Local\Temp\ipykernel_34984\2899975127.py:2: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '-' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  df['Loan_Amount_Term'].fillna('-',inplace=True)
C:\Users\Akhilesh\AppData\Local\Temp\ipykernel_34984\2899975127.py:3: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '-' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  df['Credit_History'].fillna('-',inplace=True)


In [4]:
df.fillna(0, inplace = True)

In [6]:
#!pip install SQLAlchemy==2.0.7

In [5]:
from google.cloud.sql.connector import Connector
import sqlalchemy

In [7]:
sqlalchemy.__version__

'2.0.7'

In [7]:
INSTANCE_CONNECTION_NAME = "dazzling-tensor-405719:us-central1:auto-ml" # i.e demo-project:us-central1:demo-instance
print(f"Your instance connection name is: {INSTANCE_CONNECTION_NAME}")
DB_USER = "postgres"
DB_PASS = "}gL<t,[bmnSzF-s:"
DB_NAME = "loan-app"

Your instance connection name is: dazzling-tensor-405719:us-central1:auto-ml


In [9]:
#!pip install pg8000

In [8]:
# function to return the database connection object
connector = Connector()

def getconn():
    conn = connector.connect(
        INSTANCE_CONNECTION_NAME,
        "pg8000",
        user=DB_USER,
        password=DB_PASS,
        db=DB_NAME
    )
    return conn

# create connection pool with 'creator' argument to our connection object function
pool = sqlalchemy.create_engine(
    "postgresql+pg8000://",
    creator=getconn,
)

In [9]:
import os

In [10]:
os.environ['GOOGLE_APPLICATION_CREDENTIALS'] = "dazzling-tensor-405719-b0b850808aff.json"

In [11]:
df.to_sql("loan_approvals", pool, if_exists="replace", index=False)

598

In [12]:
with pool.connect() as db_conn:
  # create ratings table in our sandwiches database
    with open("db/init.sql") as file:
        query = sqlalchemy.sql.text(file.read())
        print(query)
        db_conn.execute(query)
        db_conn.commit()


DO $$ DECLARE
rec RECORD;
BEGIN
	FOR rec IN (
	SELECT
		tablename
	FROM
		pg_tables
	WHERE
		schemaname = 'public') LOOP
EXECUTE 'DROP TABLE IF EXISTS ' || rec.tablename || ' CASCADE';
END LOOP;
END $$;

CREATE TABLE loan_approvals (
    "Loan_ID" VARCHAR,
    "Gender" VARCHAR,
    "Married" VARCHAR,
    "Dependents" VARCHAR,
    "Education" VARCHAR,
    "Self_Employed" VARCHAR,
    "ApplicantIncome" NUMERIC(6),
    "CoapplicantIncome" NUMERIC(6),
    "LoanAmount" NUMERIC(3),
    "Loan_Amount_Term" VARCHAR,
    "Credit_History" VARCHAR,
    "Property_Area" VARCHAR,
    "Loan_Status" VARCHAR
);

CREATE TABLE dim_gender (
    "Gender_ID" SERIAL PRIMARY KEY,
    "Gender" VARCHAR
);

CREATE TABLE dim_married (
    "Married_ID" SERIAL PRIMARY KEY,
    "Married" VARCHAR
);

CREATE TABLE dim_dependents (
    "Dependents_ID" SERIAL PRIMARY KEY,
    "Dependents" VARCHAR
);

CREATE TABLE dim_education (
   "Education_ID" SERIAL PRIMARY KEY,
   "Education" VARCHAR
);

CREATE TABLE dim_self_employ

In [13]:
with pool.connect() as db_conn:
    with open("db/load_data.sql") as file:
        query = sqlalchemy.sql.text(file.read())
        print(query)
        db_conn.execute(query)
        db_conn.commit()
        

--Populate Gender Dimension Table
insert
	into
	dim_gender ("Gender")
select
	distinct on
	("Gender") "Gender"
from
	loan_approvals;

--Populate Married Dimension Table
insert
	into
	dim_married ("Married")
select
	distinct on
	("Married") "Married"
from
	loan_approvals;

--Populate Dependents Dimension Table
insert
	into
	dim_dependents ("Dependents")
select
	distinct on
	("Dependents") "Dependents"
from
	loan_approvals;

--Populate Education Dimension Table
insert
	into
	dim_education ("Education")
select
	distinct on
	("Education") "Education"
from
	loan_approvals;

--Populate Self Employed Dimension Table
insert
	into
	dim_self_employed ("Self_Employed")
select
	distinct on
	("Self_Employed") "Self_Employed"
from
	loan_approvals;

--Populate Loan Amount Term Dimension Table
insert
	into
	dim_loan_amount_term ("Loan_Amount_Term")
select
	distinct on
	("Loan_Amount_Term") "Loan_Amount_Term"
from
	loan_approvals;

--Populate Property Area Dimension Table
insert
	into
	dim_property_are

In [19]:
with pool.connect() as db_conn:
    # query and fetch ratings table
    results = db_conn.execute(sqlalchemy.text("SELECT * FROM loan_approvals")).fetchall()
    # show results
    print(results)
    for row in results:
        print(row)

[]


In [57]:
!pip install sqlparse
import sqlparse


In [78]:
import re
def extract_table_names(sql_content):
    table_names = set()

    # Use regular expression to find CREATE TABLE statements
    create_table_regex = re.compile(r'CREATE TABLE\s+"?(\w+)"?\s*\(', re.IGNORECASE)

    matches = create_table_regex.findall(sql_content)
    table_names.update(matches)

    return table_names

# Replace this variable with the path to your SQL file
sql_file_path = 'db/init.sql'

# Read the SQL file
with open(sql_file_path, 'r') as sql_file:
    sql_content = sql_file.read()

# Extract table names from the SQL file
table_names = extract_table_names(sql_content)

In [79]:
table_names

{'dim_credit_history',
 'dim_dependents',
 'dim_education',
 'dim_gender',
 'dim_loan_amount_term',
 'dim_loan_status',
 'dim_married',
 'dim_property_area',
 'dim_self_employed',
 'fct_loan_approvals',
 'loan_approvals'}

In [70]:
query.get_children().

SyntaxError: invalid syntax (1305039659.py, line 1)